# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "dadosabertosreceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 07:38:23,032 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("cnpj_basico", T.StringType(), True),
    T.StructField("razao_social", T.StringType(), True),
    T.StructField("natureza_juridica", T.StringType(), True),
    T.StructField("qualificacao_responsavel", T.StringType(), True),
    T.StructField("capital_social", T.StringType(), True),
    T.StructField("porte_empresa", T.StringType(), True),
    T.StructField("ente_federativo_responsavel", T.StringType(), True),
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("dadosabertosreceita")
local = f"s3a://landing/dadosabertosreceita/{ano_mes}/empresas/*.csv"

In [5]:
config_adicional = {
    'sep': ';',
    'encoding': 'latin1'
}

# Leitura do arquivo

In [6]:
df_compliance_receita_empresas  = extrator_bronze.criar_view_temporaria_arquivo(local,'compliance_empresas', schema,**config_adicional)

2024-10-15 07:38:26,791 - INFO - Lista de yaml carregada com sucesso
2024-10-15 07:38:30,176 - INFO - Criando view temporaria.


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/dadosabertosreceita/empresas'

In [8]:
extrator_bronze.salvar_delta(df_compliance_receita_empresas, 'overwrite')

2024-10-15 07:38:30,187 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 07:39:20,889 - INFO - Dados persistidos com sucesso
2024-10-15 07:39:20,890 - INFO - s3a://bronze/dadosabertosreceita/empresas
2024-10-15 07:39:20,890 - INFO - Aguarde... Realizando optimize
2024-10-15 07:41:27,138 - INFO - Optimize executado com sucesso.
2024-10-15 07:41:27,140 - INFO - Aguarde... Realizando vacuum
2024-10-15 07:41:45,814 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-10-15 07:41:46,273 - INFO - Sessão Spark finalizada.
